# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [248]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

from scipy.stats import ks_2samp
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

%matplotlib inline

In [249]:
df = pd.read_csv('previsao_de_renda.csv')

In [250]:
df = df.dropna()
df = df.drop('Unnamed: 0', axis=1)

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na base de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [251]:
# Definindo as colunas que serão usadas como features (X) e o alvo (y)
features = ['sexo', 'tipo_renda', 'educacao', 'tempo_emprego']
alvo = 'renda'

X = df[features]
y = df[alvo]

# Separando 25% para teste e 75% para treinamento
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.25, random_state=42)

# Verificando o tamanho dos conjuntos de treinamento e teste
print("Tamanho dos conjuntos de treinamento:")
print(X_treino.shape, y_treino.shape)

print("\nTamanho dos conjuntos de teste:")
print(X_teste.shape, y_teste.shape)



Tamanho dos conjuntos de treinamento:
(9320, 4) (9320,)

Tamanho dos conjuntos de teste:
(3107, 4) (3107,)


In [252]:
# Criando uma lista com os valores de alpha
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

# criando uma lista para armazenar os resultados do R²
r2_results = []

# Iterando sobre os valores de alpha e ajustando o modelo com cada um deles
for alpha in alphas:
    modelo = 'renda ~ sexo + C(tipo_renda) + C(educacao, Treatment(2)) + tempo_emprego + 1'
    md = smf.ols(modelo, data=df)
    reg = md.fit_regularized(
        method='elastic_net',
        refit=True,
        L1_wt=0,
        alpha=alpha  #alpha porque esse valor está sendo substituído no loop for pelos valores alphas citados acima
    )
    
    # Calculando o R² na base de testes e adicionando na lista r2_results
    y_pred = reg.predict(X_teste)
    r2 = r2_score(y_teste, y_pred)
    r2_results.append(r2)

# Criando um DataFrame para visualização dos resultados
results_df = pd.DataFrame({'Alpha': alphas, 'R2': r2_results})

# Imprimindo os resultados
results_df

,Alpha,R2
0,0.000,0.296249
1,0.001,0.296271
2,0.005,0.296372
3,0.010,0.296403
4,0.050,0.293131
5,0.100,0.285581


##### Percebemos que o modelo com uma regularização ridge com alpha de 0.010 pode ser considerada melhor por ser a maior

In [253]:
# Criando uma lista com os valores de alpha
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

# criando uma lista para armazenar os resultados do R²
r2_results = []

# Iterando sobre os valores de alpha e ajustando o modelo com cada um deles
for alpha in alphas:
    modelo = 'renda ~ sexo + C(tipo_renda) + C(educacao, Treatment(2)) + tempo_emprego + 1'
    md = smf.ols(modelo, data=df)
    reg = md.fit_regularized(
        method='elastic_net',
        refit=True,
        L1_wt=1,
        alpha=alpha  #alpha porque esse valor está sendo substituído no loop for pelos valores alphas citados acima
    )
    
    # Calculando o R² na base de testes e adicionando na lista r2_results
    y_pred = reg.predict(X_teste)
    r2 = r2_score(y_teste, y_pred)
    r2_results.append(r2)

# Criando um DataFrame para visualização dos resultados
results_df = pd.DataFrame({'Alpha': alphas, 'R2': r2_results})

# Imprimindo os resultados
results_df

,Alpha,R2
0,0.000,0.296249
1,0.001,0.296249
2,0.005,0.296249
3,0.010,0.296149
4,0.050,0.296149
5,0.100,0.296149


##### Percebemos que o modelo com uma regularização Lasso com alpha até 0.005 é considerada melhor

In [254]:
# Rodando um modelo stepwise.
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out=0.05, 
                       verbose=True):
    
    included = list(initial_list)
    while True:
        changed = False
        
        # Forward step
        excluded = list(set(X.columns) - set(included))
        new_pval = {}
        for new_column in excluded:
            model = sm.OLS(y, X[included + [new_column]]).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = min(new_pval.values())
        if best_pval < threshold_in:
            best_feature = [k for k, v in new_pval.items() if v == best_pval][0]
            included.append(best_feature)
            changed = True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # Backward step
        model = sm.OLS(y, X[included]).fit()
        pvalues = model.pvalues.iloc[1:]  # Remove a constante
        worst_pval = pvalues.max()  # null if pvalues is empty
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        
        if not changed:
            break
    return included


In [255]:
#  Avaliando o  𝑅2 na base de testes.

# Realizando o one-hot encoding para todas as colunas categóricas do X_treino
X_treino = pd.get_dummies(X_treino, drop_first=True)

# Selecionando as variáveis usando Stepwise na base de treinamento
selected_features = stepwise_selection(X_treino, y_treino)

# Ajustando o modelo final usando as variáveis selecionadas na base de treinamento
modelo_final = sm.OLS(y_treino, X_treino[selected_features]).fit()

# Calculando o R² na base de testes usando as mesmas variáveis selecionadas:

# Convertendo as colunas categóricas do X_teste usando o one-hot encoding
X_teste = pd.get_dummies(X_teste, drop_first=True)

# Selecionando apenas as mesmas colunas selecionadas no selected_features
X_teste_selected = X_teste[selected_features]

# Fazendo a previsão usando o modelo final ajustado na base de treinamento
y_pred = modelo_final.predict(X_teste_selected)

# Calcule o R² na base de testes
r2 = r2_score(y_teste, y_pred)
print("R² na base de testes:", r2)


Add  sexo_M                         with p-value 0.0
Add  tempo_emprego                  with p-value 0.0
Add  educacao_Secundário            with p-value 0.000220304
Add  tipo_renda_Empresário          with p-value 0.000919524
Add  educacao_Superior incompleto   with p-value 0.000543283
R² na base de testes: 0.296016782598331


## Durante a análise dos modelos vemos:

* O R² de **0.296403** do modelo com uma regularização Ridge com alpha de 0.01.
* O R² de **0.296249** do modelo com uma regularização Lasso com alpha até 0.005.
* O R² de **0.296016** do modelo com stepwise.

Portanto o R² do modelo com uma regularização ridge com alpha de 0.010 pode ser considerado melhor.

In [256]:
# Preparando as colunas com as dummies sugeridas pelo stepwise
df['sexo_m'] = (df['sexo'] == 'M').astype(int)  
df['educacao_Secundário'] = (df['educacao'] == 'Secundário').astype(int)  
df['tipo_renda_Empresário'] = (df['tipo_renda'] == 'Empresário').astype(int)
df['educacao_Superior incompleto'] = (df['educacao'] == 'Superior incompleto').astype(int) 

# Ajustando o modelo 
modelo = 'renda ~ C(sexo_m) + C(tipo_renda_Empresário) + C(educacao_Secundário) + tempo_emprego + 0'
md = smf.ols(modelo, data=df)
reg = md.fit_regularized(
        method='elastic_net',
        refit=True,
        L1_wt=0,
        alpha=0.01
    )

# Obtendo os valores preditos do modelo ajustado
y_pred = reg.fittedvalues

# Calculando a soma total dos quadrados (SST)
sst = ((df['renda'] - df['renda'].mean()) ** 2).sum()

# Calculando a soma dos quadrados da regressão (SSR)
ssr = ((y_pred - df['renda'].mean()) ** 2).sum()

# Calculando o R²
r2 = ssr / sst
print(f"R² do modelo ajustado: {r2}")

R² do modelo ajustado: 0.25196116762805043


In [257]:
# Convertendo as variáveis categóricas em dummies
df = pd.get_dummies(df, columns=['sexo_m', 'tipo_renda_Empresário', 'educacao_Secundário'], drop_first=True)

# Separando os dados em conjunto de treinamento e teste
X = df.drop(columns=['renda', 'data_ref', 'estado_civil', 'tipo_residencia', 'sexo', 'tipo_renda','educacao'])
y = df['renda']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajustando a árvore de regressão com scikit-learn
tree_regressor = DecisionTreeRegressor()
tree_regressor.fit(X_train, y_train)

# Obtendo os valores preditos da árvore de regressão no conjunto de teste
y_pred_tree = tree_regressor.predict(X_test)

# Calculando o R² da árvore de regressão no conjunto de teste
r2_tree_test = r2_score(y_test, y_pred_tree)
print(f"R² da árvore de regressão (conjunto de teste): {r2_tree_test}")

R² da árvore de regressão (conjunto de teste): 0.27728544891221296


Ao analisar os resultados, fica evidente que o modelo com árvore de decisão apresenta um desempenho superior.